# Definition of the problem

Given $G$ as a complete weighted graph with $n$ vertices find a cycle of minimal cost:

- **Complete**: a graph in which each pair of $graph$ $vertices$ is connected by an edge.
- **Weighted**: a graph of which all of its edges has a dedicated number on them indicating the travel cost between edges.
- **Minimal cost of the cycle**: the shortest or the cheapest sum of edges to travel through the graph.

# Code

Import libraries

In [1]:
import copy # to copy graph and not to reference it
from random import randint # to generate random numbers
from math import inf as oo # to denote travel distance to the same vert
from math import sqrt
from math import isinf

# Basis

The set of vertices will be {0, 1, 2, 3, ..., n-1}

Without loss of generality, we can consider $0$ to be the start and the end point of cycles

In [2]:
MAX_DISTANCE = 100

def random_symetric_graph(n):
    ''' Random Symetric matrix of size nxn '''
    dist_matrix = [[oo for _ in range(n)] for _ in range(n)]
    for i in range(n):
        for j in range(i+1,n):
            v = randint(1, MAX_DISTANCE)
            dist_matrix[i][j] = v
            dist_matrix[j][i] = v
    return dist_matrix

def show(G):
    ''' Show adjacency matrix. Useful for debugging. '''
    n = len(G)
    r = "    "
    for i in range(n):
        r += f'{i:4}'
    r += '\n    -' + '-' *(4*n)+ '\n'
    for i in range(n):
        r += f'{i:2} | '
        for j in range(n):
            r += f'{G[i][j]:4}'
        r += '\n'
    print(r)

def cost(G, cycle):
    '''Calculate the cost of the given cycle '''
    c = 0
    n = len(G)
    for i in range(n):
        a = cycle[i]
        b = cycle[(i+1)%n]
        c += G[a][b]
    return c

# Example of the graph

G = random_symetric_graph(6)
show(G)

# Solution

### GRASP

The idea is to:
- Consider vertex $0$ as the start and the end point.
- Generate a random solution using a randomized greedy search algorithms
- Perform a local search on the temp solution
- Update the current solution with the new one if it gives a better results

**GRASP** is a multi-start or iterative metaheuristic, which utilizes iterations through two phases: construction of at least some randomized solution and a local search. Algorithm starts by constructing a randomized solution, using greedy random search, then it is necessary to apply an optimization to the solution, which in this case will be 2-opt algorithm. The 2-opt will be ran until the local minimum is found. The best solution is kept and presented as a result. (Resende and Ribeiro, 2019)

### Formal
#### Greedy Random Search

1. $H$ $\gets$ copy of $G$
2. $n$ $\gets$ len of $H$
3. $cities$ $\gets$ $list$ of a $range$ of $n$
4. $cycle$ $\gets$ $empty$ $list$
5. $city\gets 0$
6. **while** len of cities is not 0 **do**
7. $\quad$ $city$_$neighbours$$\gets$ $current$ $node$
8. $\quad$ $random$_$distance$$\gets$ $distance$ $to$ $random$ $neighbour$
9. $\quad$ **if** all neighbours cost inf **do**
10. $\qquad$ $random$_$distance$ $\gets$ $0$
11. $\qquad$ $stop$ $while$ $loop$
12. $\quad$ **end if**
13. $\quad$ $random$_$city$$\gets ramdom$_$distance$
14. $\quad$ $cycle$$\gets current city$
15. $\quad$ $remove$ $current$ $city$ $from$ $cities$ $list$
16. $\quad$ **for** all **n do**
17. $\qquad$ set current node $dist$ to $inf$
18. $\quad$ **end for**
19. $\quad$ **if** $random\_city$ $not$ 0 **do**
20. $\qquad$ $city$ $\gets$ $random\_city$
21. $\quad$ **end if**
22. **end while**
23. **return cycle, cost of G**

First while does $n$ cycles. Inside while loop we check if all of the neighbours are inf long which gives $n-1$ more computations. After that we set 1 neighbour's distance to be $inf$ long. This algorithm costs:

$$O(n\cdot (n-1) \cdot 1)=O(n^2)$$

In [3]:
def greedy_rand_search(G):
    H = copy.deepcopy(G)
    n = len(H)
    cities = list(range(n))
    cycle = []
    city = 0
    while len(cities)>0:
        city_neighbours = H[city]
        random_distance = city_neighbours[randint(0, len(city_neighbours) - 1)]
        while isinf(random_distance):
            random_distance = city_neighbours[randint(0, len(city_neighbours) - 1)]
            if all_inf(city_neighbours):
                random_distance = oo
                break
        random_city = city_neighbours.index(random_distance)
        cycle.append(city)
        cities.remove(city)
        for i in range(n):
            H[city][i] = oo
            H[i][city] = oo
        if random_city != 0:
            city = random_city
    cycle.append(0)
    return (cycle, cost(G, cycle))

def all_inf(list):
    return all(x == list[0] for x in list)

The code above illustrates the construction phase. At the each iteration update the solution by a random neighbour from the list. Keep on updating the temp solution until we have found the path to the end node. A randomized greedy construuction procedure is not always able to produce a feasable solution, thus we need to apply the repair proedure to the solution to achieve the feasibility.(Resende and Ribeiro, 2019)

### 2 opt algorithm

In [4]:
def two_opt(G, cycle):
    '''2-opt algorithm for local search'''
    size = len(cycle)
    improve = 0
        
    while improve < 100:
        '''iterates 100 times until a better cycle is found, then iterates 100 times again'''
        best_cost = cost(G, cycle)
        
        for i in range(1, size-1):
            for k in range(i+1, size):
                new_cycle = two_opt_swap(cycle, i, k)
                new_cost = cost(G, new_cycle)
                
                if new_cost < best_cost:
                    improve = 0
                    cycle = new_cycle
                    best_cost = new_cost
                    
        improve += 1
    return(cycle, best_cost)

### 2 opt swap algorithm

In [5]:
def two_opt_swap(cycle, i, k):
    '''algorithm that manipulates a given cycle'''
    size = len(cycle)
    new_cycle = list()
    
    '''append 0 to i to the new cycle'''
    for u in range(0, i):
        new_cycle.append(cycle[i])
    '''append i to k to the new cycle in reverse order'''
    dec = 0
    for o in range(i, k+1):
        new_cycle.append(cycle[k-dec])
        dec += 1
    '''append k+1 to end to the new cycle'''
    for p in range(k+1, size):
        new_cycle.append(cycle[p])
    
    return new_cycle

The **local search** phase usually improves the construction solution. It starts by iterating through the construction solution updating it with a better and more suitable neighbours for the problem to be solved. It terminates when no better solution is found in the neighbourhood.

### Testing random greedy search

G = random_symetric_graph(9)
show(G)
greedy_cycle, greedy_cost = greedy_rand_search(G)
print("greedy cycle: ", greedy_cycle)
print("greedy cost:  ", greedy_cost)

### Testing 2-opt algorithm

best_cycle, best_cost = two_opt(G, greedy_cycle)
print("best cycle: ", best_cycle)
print("best cost:  ", best_cost)

## GRASP algorithm

In [6]:
def GRASP(G):
    
    grasp_cycle = list()
    grasp_cost = oo
    iteration = 0

    while iteration < 100:

        greedy_cycle, greedy_cost = greedy_rand_search(G)
        best_cycle, best_cost = two_opt(G, greedy_cycle)

        if best_cost < grasp_cost:
            grasp_cycle = best_cycle
            grasp_cost = best_cost
            
        iteration += 1
        
    return (grasp_cycle, grasp_cost)

Lastly, the **GRASP** algorithm needs few more parameters to be set in order to start. The stopping criterion is described by the number of iterations. **Iteration** variable is defined in the code above, and is set to the number found through the trial of error which gave the best results. Although by increasing the iterations from this point most likely will not give new and better resuts, the quality of incumbent (the best current solution) may only improve. (Resende and Ribeiro, 2019)

In [7]:
G = random_symetric_graph(9)
show(G)
grasp_cycle, grasp_cost = GRASP(G)
print("Cycle: ", grasp_cycle)
print("Cost:  ", grasp_cost)

       0   1   2   3   4   5   6   7   8
    -------------------------------------
 0 |  inf  38  65  72  50  17  11   6   2
 1 |   38 inf  13  42  61  63   9  16   7
 2 |   65  13 inf  91  67  76  28  27  67
 3 |   72  42  91 inf  81  68  27  85  94
 4 |   50  61  67  81 inf  56  14  42  91
 5 |   17  63  76  68  56 inf  12  54  69
 6 |   11   9  28  27  14  12 inf  40  94
 7 |    6  16  27  85  42  54  40 inf  73
 8 |    2   7  67  94  91  69  94  73 inf

Cycle:  [0, 8, 0, 6, 5, 3, 2, 1, 7, 4]
Cost:   221


# References

* Weisstein, E.W. (n.d.).
**Complete Graph. [online] mathworld.wolfram.com.** 
Available at: https://mathworld.wolfram.com/CompleteGraph.html [Accessed 18 Mar. 2020].

* 2-Opt Traveling Salesman Java | Technical-Recipes.Com (2017) 
available from <https://www.technical-recipes.com/2017/applying-the-2-opt-algorithm-to-traveling-salesman-problems-in-java/> [18 March 2020]

* Resende, M. and Ribeiro, C. (n.d.). Greedy Randomized Adaptive Search Procedures: Advances and Extensions. [online] Available at: http://www.dcc.ic.uff.br/~celso/artigos/resende-ribeiro-GRASP-HMH3.pdf [Accessed 26 Mar. 2020].